In [ ]:
"""
Gaia データセットの欠損値チェックとバリデーション

このスクリプトは、Gaiaデータセットの品質を確認し、
有効なデータのみを抽出するためのユーティリティ関数を提供します。
"""

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_parquet('../dataset/train-00000-of-00001.parquet')

In [ ]:
import pandas as pd

def check_missing_values_fast(df):
    print("=== 欠損値の確認 ===")
    print(f"\n全データ数: {len(df):,}")

    target_groups = {
        'astrometry': ['ra', 'dec', 'parallax', 'parallax_error'],
        'photometry': ['phot_g_mean_mag']
    }

    for parent_col, sub_cols in target_groups.items():
        print(f"\n--- {parent_col} ---")
        if parent_col not in df.columns:
            print(f"Skipping: {parent_col} not found.")
            continue
            
        # 【重要】辞書の列を一時的にDataFrameに展開（これが高速化の肝です）
        # tolist()でPythonのリストにしてからDataFrame化すると非常に高速です
        temp_df = pd.DataFrame(df[parent_col].tolist())
        
        # 指定されたカラムのみ抽出（存在しないキーはNaNになるので安全）
        # temp_dfに存在しないカラムがリクエストされた場合のハンドリング
        existing_cols = [c for c in sub_cols if c in temp_df.columns]
        missing_cols_in_data = set(sub_cols) - set(existing_cols)

        # 一括で欠損値を計算
        if not existing_cols:
            print("  (指定されたサブカラムがデータ内に存在しません)")
        else:
            null_counts = temp_df[existing_cols].isna().sum()
            
            for col in sub_cols:
                if col in missing_cols_in_data:
                    # キー自体が存在しない＝全件欠損扱い
                    count = len(df)
                else:
                    count = null_counts[col]
                
                percent = (count / len(df)) * 100
                print(f"{col:<20}: {count:>8,}件 ({percent:>6.2f}%)")

# 実行
check_missing_values_fast(df)

=== 欠損値の確認 ===

全データ数: 100,000

--- astrometry ---
ra                  :        0件 (  0.00%)
dec                 :        0件 (  0.00%)
parallax            :      613件 (  0.61%)
parallax_error      :      613件 (  0.61%)

--- photometry ---
phot_g_mean_mag     :        1件 (  0.00%)


In [ ]:
def check_data_quality(df):
    """
    視差データの品質を確認する
    
    Parameters:
    -----------
    df : pd.DataFrame
        Gaiaデータセット
    """
    print("\n=== 視差データの品質確認 ===")
    parallax = df['astrometry'].apply(lambda x: x.get('parallax') if isinstance(x, dict) else None)
    parallax_error = df['astrometry'].apply(lambda x: x.get('parallax_error') if isinstance(x, dict) else None)
    
    # 基本統計量
    print(f"\n視差の統計:")
    print(f"  平均: {parallax.mean():.3f} mas")
    print(f"  中央値: {parallax.median():.3f} mas")
    print(f"  最小値: {parallax.min():.3f} mas")
    print(f"  最大値: {parallax.max():.3f} mas")
    print(f"  標準偏差: {parallax.std():.3f} mas")
    
    # 負の視差の確認
    negative_parallax = (parallax < 0).sum()
    print(f"\n負の視差を持つデータ: {negative_parallax:,}件 ({negative_parallax/len(df)*100:.2f}%)")
    
    # 視差エラーが視差より大きいデータ
    large_error = (parallax_error > parallax).sum()
    print(f"視差エラーが視差より大きいデータ: {large_error:,}件 ({large_error/len(df)*100:.2f}%)")
    
    # Signal-to-Noise比が低いデータ（視差/エラー < 3）
    low_snr = (parallax / parallax_error < 3).sum()
    print(f"S/N比が3未満のデータ: {low_snr:,}件 ({low_snr/len(df)*100:.2f}%)")

check_data_quality(df)


=== 視差データの品質確認 ===

視差の統計:
  平均: 1.111 mas
  中央値: 0.697 mas
  最小値: -4.879 mas
  最大値: 130.853 mas
  標準偏差: 1.530 mas

負の視差を持つデータ: 813件 (0.81%)
視差エラーが視差より大きいデータ: 1,873件 (1.87%)
S/N比が3未満のデータ: 7,131件 (7.13%)


In [ ]:
def validate_gaia_data(df, verbose=True):
    """
    Gaiaデータの品質チェックと有効なデータの抽出
    
    Parameters:
    -----------
    df : pd.DataFrame
        Gaiaデータセット
    verbose : bool, default=True
        詳細なレポートを表示するかどうか
        
    Returns:
    --------
    df_valid : pd.DataFrame
        有効なデータのみを含むDataFrame
    validation_report : dict
        バリデーション結果のレポート
    """
    
    validation_report = {
        'total_records': len(df),
        'removed': {}
    }
    
    # 初期データ
    df_valid = df.copy()
    initial_count = len(df_valid)
    
    # 1. 必須フィールドの欠損値チェック
    required_fields = [
        ('astrometry', 'ra'),
        ('astrometry', 'dec'),
        ('astrometry', 'parallax'),
        ('astrometry', 'parallax_error'),
        ('photometry', 'phot_g_mean_mag')
    ]
    
    for parent, field in required_fields:
        null_mask = df_valid[parent].apply(lambda x: pd.isna(x.get(field)) if isinstance(x, dict) else True)
        null_count = null_mask.sum()
        if null_count > 0:
            validation_report['removed'][f'{parent}.{field}_null'] = null_count
            df_valid = df_valid[~null_mask].copy()
    
    # 2. 視差が正の値のみ
    negative_parallax_mask = df_valid['astrometry'].apply(lambda x: x.get('parallax', 0) if isinstance(x, dict) else 0) <= 0
    negative_count = negative_parallax_mask.sum()
    if negative_count > 0:
        validation_report['removed']['negative_or_zero_parallax'] = negative_count
        df_valid = df_valid[~negative_parallax_mask].copy()
    
    # 3. 視差の精度チェック（視差 > 3 * 視差エラー）
    # S/N比が3以上のデータのみを使用
    parallax_values = df_valid['astrometry'].apply(lambda x: x.get('parallax', 0) if isinstance(x, dict) else 0)
    parallax_error_values = df_valid['astrometry'].apply(lambda x: x.get('parallax_error', float('inf')) if isinstance(x, dict) else float('inf'))
    low_precision_mask = parallax_values <= 3 * parallax_error_values
    low_precision_count = low_precision_mask.sum()
    if low_precision_count > 0:
        validation_report['removed']['low_precision_parallax'] = low_precision_count
        df_valid = df_valid[~low_precision_mask].copy()
    
    # 4. 異常値のチェック（視差が極端に大きい/小さい）
    # 視差が1000 mas（距離1pc）以上 または 0.1 mas（距離10kpc）以下を除外
    parallax_values = df_valid['astrometry'].apply(lambda x: x.get('parallax', 0) if isinstance(x, dict) else 0)
    extreme_parallax_mask = (parallax_values > 1000) | (parallax_values < 0.1)
    extreme_count = extreme_parallax_mask.sum()
    if extreme_count > 0:
        validation_report['removed']['extreme_parallax'] = extreme_count
        df_valid = df_valid[~extreme_parallax_mask].copy()
    
    validation_report['valid_records'] = len(df_valid)
    validation_report['total_removed'] = initial_count - len(df_valid)
    validation_report['retention_rate'] = len(df_valid) / initial_count * 100 if initial_count > 0 else 0
    
    if verbose:
        print_validation_report(validation_report)
    
    return df_valid, validation_report

df_valid, report = validate_gaia_data(df)
print(f"有効なデータ数: {len(df_valid)}")


データバリデーション結果
総データ数: 100,000件
有効データ数: 92,156件
除外データ数: 7,844件
データ保持率: 92.16%

除外理由の内訳:
  - astrometry.parallax_null: 613件 (0.61%)
  - photometry.phot_g_mean_mag_null: 1件 (0.00%)
  - negative_or_zero_parallax: 813件 (0.81%)
  - low_precision_parallax: 6,318件 (6.32%)
  - extreme_parallax: 99件 (0.10%)
有効なデータ数: 92156


In [ ]:
def print_validation_report(report):
    """
    バリデーション結果のレポートを表示する
    
    Parameters:
    -----------
    report : dict
        バリデーション結果のレポート
    """
    print("\n" + "="*50)
    print("データバリデーション結果")
    print("="*50)
    print(f"総データ数: {report['total_records']:,}件")
    print(f"有効データ数: {report['valid_records']:,}件")
    print(f"除外データ数: {report['total_removed']:,}件")
    print(f"データ保持率: {report['retention_rate']:.2f}%")
    
    if report['removed']:
        print("\n除外理由の内訳:")
        for reason, count in report['removed'].items():
            percentage = (count / report['total_records']) * 100
            print(f"  - {reason}: {count:,}件 ({percentage:.2f}%)")
    print("="*50)

print_validation_report(report)

In [ ]:
def get_quality_summary(df):
    """
    データセット全体の品質サマリーを取得する
    
    Parameters:
    -----------
    df : pd.DataFrame
        Gaiaデータセット
        
    Returns:
    --------
    summary : dict
        品質サマリー
    """
    parallax = df['astrometry']['parallax']
    parallax_error = df['astrometry']['parallax_error']
    
    summary = {
        'total_count': len(df),
        'parallax_stats': {
            'mean': float(parallax.mean()),
            'median': float(parallax.median()),
            'std': float(parallax.std()),
            'min': float(parallax.min()),
            'max': float(parallax.max())
        },
        'quality_issues': {
            'missing_parallax': int(parallax.isna().sum()),
            'negative_parallax': int((parallax < 0).sum()),
            'low_snr': int((parallax / parallax_error < 3).sum()),
            'extreme_values': int(((parallax > 1000) | (parallax < 0.1)).sum())
        }
    }
    
    return summary